# Scenarios
In this notebook we'll show how to run different scenario's on your portfolio, to see what a certain action would do to your temperature score.

### Setting up
We'll set up the imports, data providers, and load the portfolio. For more details about this, please have a look at notebook one (getting started).

In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetValidation
%aimport -pandas
import pandas as pd

In [2]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")
companies = portfolio[["company_id", "company_name"]].to_dict(orient="records")

company_data = SBTi.data.get_company_data([provider], companies)
target_data = SBTi.data.get_targets([provider], companies)
company_data = pd.merge(left=company_data,
                        right=portfolio[
                            [column
                                for column in portfolio.columns
                                if column not in ["company_name"]]],
                        how="left",
                        on=["company_id"])

target_validation = TargetValidation(target_data, company_data)
portfolio_data = target_validation.target_validation()

## Base scenario
To determine what the effect is of running a certain scenario, we first need a base line. To do so, we run the calculate method of the temperature score, without defining a scenario. We'll determine the effect of our scenario's by looking at the aggregated temperature score for the short timeframe in the combined s1s2s3 scope. Of course you can use multiple, or different criteria.

In [3]:
temperature_score = TemperatureScore()
base_scenario = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario, PortfolioAggregationMethod.WATS, [])
base_scenario_aggregated["short"]["s1s2s3"]["all"]["score"]

2.0568

## Scenario 1
In scenario 1 we engage companies to set targets. This means that the default score (i.e. the score when a company didn't set a target) will be set to 2 degrees celcius.

In [4]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [5]:
temperature_score = TemperatureScore()
temperature_score.set_scenario(scenario)
scenario_one = temperature_score.calculate(portfolio_data.copy())
scenario_one = temperature_score.aggregate_scores(scenario_one, PortfolioAggregationMethod.WATS, [])
scenario_one["short"]["s1s2s3"]["all"]["score"]

1.2048

## Scenario 2
In scenario 2 we engage companies to have their targets validated by the SBTi. This means that the maximum score for all targets will be capped at 1.75 degrees celcius.

In [6]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.APPROVED_TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [7]:
temperature_score = TemperatureScore()
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_two = temperature_score.aggregate_scores(scenario_two, PortfolioAggregationMethod.WATS, [])
scenario_two["short"]["s1s2s3"]["all"]["score"]

1.012

## Scenario 3a
In scenario 3a we engage the top 10 contributors to set targets. This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees celcius.

In [8]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [9]:
temperature_score = TemperatureScore()
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_two = temperature_score.aggregate_scores(scenario_two, PortfolioAggregationMethod.WATS, [])
scenario_two["short"]["s1s2s3"]["all"]["score"]

1.7667

## Scenario 3b
In scenario 3b we engage the top 10 contributors to have their targets validated by the SBTi. This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees celcius.

In [10]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [11]:
temperature_score = TemperatureScore()
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_two = temperature_score.aggregate_scores(scenario_two, PortfolioAggregationMethod.WATS, [])
scenario_two["short"]["s1s2s3"]["all"]["score"]

1.7063

## Scenario 4a
In scenario 4a we engage the predefined companoies to set targets. This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees celcius.

In [12]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [13]:
temperature_score = TemperatureScore()
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_two = temperature_score.aggregate_scores(scenario_two, PortfolioAggregationMethod.WATS, [])
scenario_two["short"]["s1s2s3"]["all"]["score"]

1.9704

## Scenario 4b
In scenario 4b we engage the predefined companies to have their targets validated by the SBTi. This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees celcius.

In [14]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [15]:
temperature_score = TemperatureScore()
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
scenario_two = temperature_score.aggregate_scores(scenario_two, PortfolioAggregationMethod.WATS, [])
scenario_two["short"]["s1s2s3"]["all"]["score"]

1.9512